In [62]:
from konlpy.tag import Okt
from tqdm import tqdm
import pandas as pd
from datetime import timedelta

In [63]:
# import re

# def preprocess_text(text):
#     # 괄호와 괄호 안에 있는 내용 삭제
#     text = re.sub(r'\([^()]*\)', '', text)  # () 안에 있는 내용 삭제
#     text = re.sub(r'\[[^\[\]]*\]', '', text)  # [] 안에 있는 내용 삭제
#     text = re.sub(r'\{[^{}]*\}', '', text)  # {} 안에 있는 내용 삭제

#     pattern_others = re.compile(r'\.([^\.]*(?:기자|특파원|교수|작가|대표|논설|고문|주필|부문장|팀장|장관|원장|연구원|이사장|위원|실장|차장|부장|에세이|화백|사설|소장|단장|과장|기획자|큐레이터|저작권|평론가|©|©|ⓒ|\@|\/|=|▶|무단|전재|재배포|금지|\[|\]|\(\))[^\.]*)$')
#     text = pattern_others.sub('.', text)

#     # 특수문자 제거
#     text = re.sub(r'[^%.,+-\w\s]', ' ', text)  # 특수문자를 공백으로 대체

#     # 공백 제거
#     text = re.sub(r'\s+', ' ', text)  # 연속된 공백을 하나의 공백으로 대체
#     text = text.strip()  # 문장 앞뒤의 공백 제거

#     # 연속된 줄바꿈을 하나의 줄바꿈으로 대체
#     text = re.sub(r'\n+', '\n', text)

#     return text

In [64]:
dh_df = pd.read_excel('./DH(뉴스+타겟).xlsx', index_col=0)

In [65]:
import re

def preprocess_text(text):
    # Delete parentheses and contents inside parentheses
    text = re.sub(r'\([^()]*\)', '', text) # 괄호와 괄호 안의 내용 삭제
    text = re.sub(r'\[[^\[\]]*\]', '', text) # 대괄호와 대괄호 안의 내용 삭제
    text = re.sub(r'\{[^{}]*\}', '', text) # 중괄호와 중괄호 안의 내용 삭제

    pattern_others = re.compile(r'\.([^\.]*(?: 기자|특파원|교수|작가|대표|논설|고문|주필|부문장|팀장|장관|원장|연구원|이사장|위원|실장|차장|부장|에세이|화백|사설|소장|단장|과장|기획자|큐레이터|저작권|평론가|©|©|ⓒ|\@|\/|=|▶|무단|전재|재배포|금지|\[|\]|\(\))[^\.]*)$')
    text = pattern_others.sub('.', text)

    # 특수 문자 제거
    text = re.sub(r'[^%.,+\-\w\s]', ' ', text) # 특수 문자를 공백으로 대체

    # 공백 정리
    text = re.sub(r'\s+', ' ', text) # 연속된 공백을 단일 공백으로 대체
    text = text.strip() # 앞뒤 공백 제거

    # 연속된 줄바꿈을 단일 줄바꿈으로 대체
    text = re.sub(r'\n+', '\n', text)

    return text

In [66]:
stopwords = ['원본보기', '아이콘', 'AD','자료','ADVERTISEMENT','Advertisement','viewer']

In [67]:
def remove_stopwords(text, stopwords):
    words = text.split()
    words = [word for word in words if word.lower() not in stopwords]
    return ' '.join(words)

In [68]:
dh_df['article_text'] = dh_df['article_text'].apply(preprocess_text)

In [69]:
dh_df['article_text'] = dh_df['article_text'].apply(remove_stopwords, stopwords=stopwords)

In [70]:
dh_df['oil_news'] = dh_df['oil_news'].apply(preprocess_text)
dh_df['oil_news'] = dh_df['oil_news'].apply(remove_stopwords, stopwords=stopwords)

In [71]:
dh_df.head()

,predict_date,oil_news,article_text,대한종가,대한타겟
0,2022-01-02,코로나19가 2년가까이 지속되면서 대면서비스업을 중심으로 한 내수경기의 부진은 더욱...,2022년에도 국제선 여객 정상화는 어렵다는 판단이며 이에 따라 항공화물 시황은 호...,29500,0
1,2022-01-03,2021년 12월 충청지역 소비자물가동향 충북과 충남의 물가 상승률이 4%대를 기록...,26일 오후 경기도 의정부체육관에서 열린 2021 의정부 도드람컵 프로배구대회 순위...,29500,0
2,2022-01-04,서울 시내 주유소 모습. 주요 에너지기관들은 2022년에도 국제 유가가 상승세를 이...,사천공항 경남도는 진에어가 오는 28일부터 김포-사천 노선에 신규 취항한다고 4일 ...,29100,0
3,2022-01-05,"저작권자 서울경제, 무단 전재 및 재배포 금지 미국 뉴욕증시의 주요 지수가 국채금리...",인천 중구 영종국제도시 대한항공 항공정비단지 부지. 인천시 제공 대한항공이 인천 영...,28900,0
4,2022-01-06,美 남미 등 수입처 다변화 안정적 물량 확보는 어려워 코로나19 여파로 원유 가격이...,"인천시-대한항공, 항공정비산업 발전 업무협약 체결 대한항공, 3346억 투자 202...",28450,0


In [36]:
n_ = []
okt = Okt()
# data = pd.read_excel('./DH(뉴스+타겟).xlsx')
data = dh_df
data['oil_news'] = data['oil_news'].astype(str)
data['oil_news'] = data['oil_news'].str.replace(r'[\(\[].*?[\)\]]', '', regex=True)
for i in tqdm(range(len(data))):
    # 바뀌어진 기사 제목에서 okt로 명사만 뽑기
    n_.append(' '.join(okt.nouns(data.loc[i, 'oil_news'])))

data['oil_nouns'] = n_
data = data[data['oil_nouns'] != '']

# 한 글자 명사 삭제
data['oil_nouns'] = data['oil_nouns'].apply(lambda x: [word for word in x.split() if len(word) > 1])
data['oil_nouns'] = data['oil_nouns'].apply(lambda x: ' '.join(x))

data['oil_nouns'] = data['oil_nouns'].str.replace('[', '').str.replace(']', '')

# data['일자'] = pd.to_datetime(data['일자'].astype(str))

# data['주식일자'] = data['일자'] + timedelta(days=1)

100%|██████████| 484/484 [00:51<00:00,  9.37it/s]


In [38]:
data.head()

,predict_date,oil_news,article_text,대한종가,대한타겟,oil_nouns
0,2022-01-02,코로나19가 2년가까이 지속되면서 대면서비스업을 중심으로 한 내수경기의 부진은 더욱...,2022년에도 국제선 여객 정상화는 어렵다는 판단이며 이에 따라 항공화물 시황은 호...,29500,0,코로나 지속 대면 서비스업 중심 경기 부진 더욱 반면 대외 무역 사상 최대 실적 기...
1,2022-01-03,2021년 12월 충청지역 소비자물가동향 충북과 충남의 물가 상승률이 4%대를 기록...,26일 오후 경기도 의정부체육관에서 열린 2021 의정부 도드람컵 프로배구대회 순위...,29500,0,충청 지역 소비자 물가 동향 충북 충남 물가 승률 기록 축산물 가격 국제 유가 계속...
2,2022-01-04,서울 시내 주유소 모습. 주요 에너지기관들은 2022년에도 국제 유가가 상승세를 이...,사천공항 경남도는 진에어가 오는 28일부터 김포-사천 노선에 신규 취항한다고 4일 ...,29100,0,서울 시내 주유소 모습 주요 에너지 기관 국제 유가 상승세 전망 코로나 바이러스 오...
3,2022-01-05,"저작권자 서울경제, 무단 전재 및 재배포 금지 미국 뉴욕증시의 주요 지수가 국채금리...",인천 중구 영종국제도시 대한항공 항공정비단지 부지. 인천시 제공 대한항공이 인천 영...,28900,0,저작권 서울 경제 무단 배포 금지 미국 뉴욕증시 주요 지수 국채 금리 상승 결과 경...
4,2022-01-06,美 남미 등 수입처 다변화 안정적 물량 확보는 어려워 코로나19 여파로 원유 가격이...,"인천시-대한항공, 항공정비산업 발전 업무협약 체결 대한항공, 3346억 투자 202...",28450,0,남미 입처 다변 안정 물량 확보 코로나 여파 원유 가격 지난해 연말 이후 상승세 국...


In [39]:
n_ = []
okt = Okt()
# data = pd.read_excel('./DH(뉴스+타겟).xlsx')
data = dh_df
data['article_text'] = data['article_text'].astype(str)
data['article_text'] = data['article_text'].str.replace(r'[\(\[].*?[\)\]]', '', regex=True)
for i in tqdm(range(len(data))):
    # 바뀌어진 기사 제목에서 okt로 명사만 뽑기
    n_.append(' '.join(okt.nouns(data.loc[i, 'article_text'])))

data['dh_nouns'] = n_
data = data[data['dh_nouns'] != '']

# 한 글자 명사 삭제
data['dh_nouns'] = data['dh_nouns'].apply(lambda x: [word for word in x.split() if len(word) > 1])
data['dh_nouns'] = data['dh_nouns'].apply(lambda x: ' '.join(x))

data['dh_nouns'] = data['dh_nouns'].str.replace('[', '').str.replace(']', '')

# data['일자'] = pd.to_datetime(data['일자'].astype(str))

# data['주식일자'] = data['일자'] + timedelta(days=1)

100%|██████████| 484/484 [01:07<00:00,  7.15it/s]


In [41]:
# data.to_excel('./대한항공(명사화).xlsx')

In [44]:
dh_nouns_df = data[['predict_date', 'oil_nouns', 'dh_nouns', '대한타겟']]

In [47]:
dh_nouns_df.to_excel('./대한항공(명사화).xlsx')

In [52]:
import pandas as pd
from collections import Counter

def count_word_frequency_dataframe(df, text_column):
    # 데이터프레임의 특정 열에서 텍스트 추출
    text = ' '.join(df[text_column].tolist())

    # 텍스트를 단어로 분할
    words = text.split()

    # 단어의 빈도를 계산
    word_frequency = Counter(words)

    # 문자 수가 한 개인 단어 제외
    word_frequency = {word: freq for word, freq in word_frequency.items() if len(word) > 1}

    # 결과를 데이터프레임으로 변환
    result_df = pd.DataFrame.from_dict(word_frequency, orient='index', columns=['빈도'])
    result_df.index.name = '단어'
    result_df.reset_index(inplace=True)

    # 빈도순으로 내림차순 정렬
    result_df = result_df.sort_values(by='빈도', ascending=False)

    # 인덱스 재설정
    result_df.reset_index(drop=True, inplace=True)

    return result_df

# # 예시 데이터프레임
# data = {
#     '문장': ['안녕하세요. 만나서 반갑습니다.', '안녕하세요. 좋은 하루 되세요.']
# }
# df = pd.DataFrame(data)

# 특정 열에서 단어 빈도 계산 및 결과 출력
result = count_word_frequency_dataframe(dh_nouns_df, 'oil_nouns')

# 빈도순으로 내림차순 정렬된 결과 데이터프레임 출력
print(result)

      단어    빈도
0     가격  4450
1     유가  4423
2     국제  3302
3     물가  2900
4     상승  2618
...   ..   ...
9051  온전     1
9052  불똥     1
9053  오락     1
9054  씨젠     1
9055  트위     1

[9056 rows x 2 columns]


In [60]:
result.loc[result['빈도'] > 10]

,단어,빈도
0,가격,4450
1,유가,4423
2,국제,3302
3,물가,2900
4,상승,2618
...,...,...
2062,합계,11
2063,반드시,11
2064,매달,11
2065,운전,11


In [73]:
dh_df.to_excel('./DH(뉴스+타겟)222.xlsx')